# IV - Gradient-based free form optimization with adaptive mesh deformation

- We noticed in [Notebook III](III-Gradient-based_optimization_control_points.ipynb) that the more control points, the best performance
- The computation time is almost the same dueto the efficient shape derivative evaluation with adjoint method
- Ideal case would be to have an infinite number of control points (or at least as much as allowed by the discretization)
- This is possible using non-parametric "free form" optimization, using adaptive mesh deformation

In [65]:
from shapeOptInductor import gen_mesh, Id, Curl
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt

___________________________
## 1 - Geometry and meshing

We first define the inductor geometry and generate the mesh.

In [66]:
# Geometric parameters
lz = 1e-2               # z-thickness of the inductor  
s = 4                   # symmetry factor (model 1/4th of the total space)

# Mesh generation
maxh = 2e-3                                             # Maximum size of mesh element
mesh = gen_mesh(airgap=4.11e-3, maxh = maxh)    # Generate the mesh and return the coordinates of control points on the first and second leg

# Define the characteristics functions
XiAir = mesh.MaterialCF({"air": 1})
XiCore = mesh.MaterialCF({"core": 1})
XiCoil = mesh.MaterialCF({"coil": 1})
materials_regions = 1 * XiAir + 2 * XiCoil + 3 * XiCore

# Draw the geometry and mesh
Draw(materials_regions, mesh, radius=0.02)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

___________________________
## 2 - State problem (time-harmonic magnetics)

### 2.a - Parameters definition

We describe the magnetic permeabilities and the source currents.

In [67]:
# Electric current (assumed sinusoidal)
f = 5e4                                                         # Working frequency (Hz)
omega = 2 * np.pi * f                                           # rad/s
nb_turn = 200                                                   # Number of turn in the coil (no unit)
I = 2                                                           # Amplitude of the source current (A)
j = nb_turn / 2 * I / (ngs.Integrate(XiCoil, mesh)) * XiCoil    # Amplitude of the source current density  (A/m²)

# Magnetic permeabilities
mu0 = 4e-7 * np.pi                                              # Void permeability (H/m)
mur = 1000                                                      # Relative permeability of iron (no unit)
mu_iron = mur * mu0                                             # Iron permeability (H/m)
delta = 0.1                                                     # Loss angle associated with the coil (rad)
mu_coil = np.exp(-1j * delta) * mu0                             # Complex permeability of the coil

### 2.b - Definition of the time-harmonic magnetics problem

Then we can write the weak formulation and solve the following problem :
$$\text{Find } \underline{a_h} \in H := \{\underline{a} \in H^1(\Omega) |\, \underline{a} = 0 \text{ on } \Gamma_D\} \quad \text{ s.t. } \quad
    \forall \underline{v} \in H, \int_\Omega \underline{\nu_h} (\nabla \underline{a_h}) \cdot (\nabla \underline{v})^* \,\mathrm{d}x = \int_{\Omega_{c, h}} j \underline{v}^* \, \mathrm{d}x. $$

In [68]:
def magWeakFormComplex(ah : ngs.FESpace.TrialFunction, 
                       v  : ngs.FESpace.TestFunction
                       ) -> tuple[ngs.BilinearForm, ngs.LinearForm]:
    """Return the sybolic weak form of the magnetic problem, 
    i.e, bilinear and linear forms (not assembled yet)."""

    # Bilinear form (transmission)
    bf =  1 / mu_iron * ngs.grad(ah) * ngs.grad(v) * ngs.dx("core")
    bf += 1 / mu_coil * ngs.grad(ah) * ngs.grad(v) * ngs.dx("coil")
    bf += 1 / mu0 * ngs.grad(ah) * ngs.grad(v) * ngs.dx("air")

    # Linear form (source)
    lf = j * v * ngs.dx("coil")

    return bf, lf


def solveStateComplex(mesh : ngs.Mesh
                      ) -> tuple[ngs.GridFunction, ngs.la.BaseMatrix]:
    """ Solve the time-harmonic magnetics problem """

    # Definition of function space
    fes = ngs.H1(mesh, order=1, dirichlet="d|e|f|g", complex=True)
    a, v = fes.TnT()

    # Definition of weak form
    bf, f = magWeakFormComplex(a, v)

    # Assembly
    K = ngs.BilinearForm(bf).Assemble()
    F = ngs.LinearForm(f).Assemble()

    # Solving
    gf = ngs.GridFunction(fes)
    Kinv = K.mat.Inverse(freedofs=fes.FreeDofs())
    gf.vec.data = Kinv * F.vec

    return gf, Kinv  # returning Kinv speed up adjoint calculation

# Example :
state, Kinv = solveStateComplex(mesh)
Draw(state.real, mesh, radius=0.02, settings = {"Objects" : { "Wireframe" : False }})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

BaseWebGuiScene

### 2.c - Definition of the post-processed quantities

We define the losses and inductance 
$$
\begin{array}{lcl}
P(\underline{a}) &=& \displaystyle  s l_z  \pi f \int_{\Omega_c} \text{Im} \left( \frac{1}{\underline{\mu_c}} \right ) |\nabla \underline{a}|^2 \;\mathrm{d}x,\\
L(\underline{a}) &=& \displaystyle \frac{s l_z}{I^2} \int_\Omega \text{Re} \left( \underline{\nu} \right) |\nabla \underline{a}|^2 \;\mathrm{d}x,\\
\end{array}$$

with their directional derivatives.

In [69]:
def L(state : ngs.GridFunction, 
      mesh : ngs.Mesh
      ) -> float:
    """ Compute inductance from the time-harmonic solution """
    nu = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return s * lz / (I**2) * ngs.Integrate(nu.real * ngs.Norm(ngs.grad(state)) ** 2, mesh)


def ddL(state : ngs.GridFunction, 
        v : ngs.FESpace.TestFunction
        ) -> ngs.comp.SumOfIntegrals:
    """ Compute directional derivative of the inductance from the time-harmonic solution """
    nu = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return 2 * s * lz / (I**2) * ngs.InnerProduct(nu.real * ngs.grad(state), ngs.grad(v)) * ngs.dx


def P(state : ngs.GridFunction,
      mesh : ngs.Mesh
      ) -> float:
    """ Compute AC losses in the conductor from the time-harmonic solution """
    nu = XiCoil / mu_coil
    return s * np.pi * f * lz * ngs.Integrate(nu.imag * ngs.Norm(ngs.grad(state)) ** 2, mesh)


def ddP(state : ngs.GridFunction, 
        v : ngs.FESpace.TestFunction
        ) -> ngs.comp.SumOfIntegrals:
    """ Compute directional derivative of AC losses in the conductor from the time-harmonic solution """
    nu = XiCoil / mu_coil
    return 2 * s * np.pi * f * lz * ngs.InnerProduct(nu.imag * ngs.grad(state), ngs.grad(v)) * ngs.dx

# Example :
losses = P(state, mesh)
inductance = L(state, mesh)
print(f"Inductance : {inductance *1000:.3} mH")
print(f"Losses : {losses:.3} W")

Inductance : 1.0 mH
Losses : 13.2 W


___________________________
## 3 - Adjoint problem

The adjoint method is an efficient way to compute derivatives of PDE-constrained problem; it requires solving one linear auxiliary problem per function $f$ to derivate that reads

$$ \text{Find } \underline{p}_f \in H \quad \text{ s.t. } \quad
    \forall \underline{v} \in H,  \quad \int_\Omega \nabla \underline{p}_f \cdot \underline{\nu}^* \nabla \underline{v} \; \mathrm{d}x = \int f'(\underline{a})(\underline{v})\; \mathrm{d}x, $$

with $f'$ the directional derivative of $f$, such that $F(\underline{a}) = \int_\Omega f(\underline{a}) \; \mathrm{d}x$.

In [70]:
def solveAdjoint(state : ngs.GridFunction,      
                 df : ngs.comp.SumOfIntegrals,
                 Kinv : ngs.la.BaseMatrix
                 ) -> ngs.GridFunction:
    """Solve the adjoint equation for a given directional derivative (right-hand side)"""

    fes = state.space
    v = fes.TestFunction()

    # Assemble the right-hand side
    f = ngs.LinearForm(-1. * df(state, v)).Assemble()
    
    # Solve using the adjoint (transconjugate) of provided FEM matrix
    gf = ngs.GridFunction(fes)
    gf.vec.data = Kinv.H * f.vec

    return gf

# Example :
adjoint_P = solveAdjoint(state, ddP, Kinv)
Draw(adjoint_P.real, mesh, radius = 0.02, settings = {"Objects" : { "Wireframe" : False }})

adjoint_L = solveAdjoint(state, ddL, Kinv)
Draw(adjoint_L.real, mesh, radius = 0.02, settings = {"Objects" : { "Wireframe" : False }})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

BaseWebGuiScene

___________________________
## 4 - Continuous shape derivatives and regularized descent direction

### 4.a - Shape derivatives

The shape derivative of the losses is a linear form obtained by Céa's method that reads
$$\mathrm{d} P(\Omega)(\phi) = \text{Re}\left( s l_z \pi f \int_{\Omega_c} \text{Im}\left(\frac{1}{\underline{\mu_c}}\right) A_\phi \nabla \underline a \cdot (\nabla \underline a)^* \; \mathrm{d} x + \int_\Omega \underline{\nu} A_\phi \nabla \underline{a} \cdot \nabla \underline p_P^* \mathrm{d} x - \int_{\Omega_c} j\, \text{div} \phi\, \underline p_P^*  \mathrm{d} x \right)  $$

with $\phi\in W^{1, \infty}(\mathbb{R}^2, \mathbb{R}^2)$ and $A_\phi = \text{div} \phi \, I_2 - \nabla \phi - \nabla \phi^\top$.

In [71]:
def sdP(state : ngs.GridFunction,
        adjoint_P : ngs.GridFunction
        ) -> ngs.LinearForm :
    """ Compute the shape derivative (linear form) of the AC losses inside the coil """

    # Finite element spaces
    mesh = state.space.mesh
    FESReal = ngs.VectorH1(mesh)
    FEScomplex = ngs.VectorH1(mesh, complex=True)
    phi = FEScomplex.TestFunction()

    # Shorthands
    nu = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    coeff_losses = s * np.pi * f * lz * (1 / mu_coil).imag
    dA = ngs.div(phi) * Id - ngs.grad(phi) - ngs.grad(phi).trans

    # Shape derivative
    shape_derivative_cplx = ngs.LinearForm(FEScomplex)
    shape_derivative_cplx += coeff_losses * ngs.InnerProduct(dA * ngs.grad(state), ngs.grad(state)) * ngs.dx("coil")
    shape_derivative_cplx += -j * XiCoil * ngs.div(phi) * adjoint_P * ngs.dx
    shape_derivative_cplx += ngs.InnerProduct(nu * dA * ngs.grad(state), ngs.grad(adjoint_P)) * ngs.dx
    shape_derivative_cplx.Assemble()

    # Take the real part
    shape_derivative = ngs.LinearForm(FESReal)
    shape_derivative.vec.FV().NumPy()[:] = np.real(shape_derivative_cplx.vec.FV().NumPy()[:])

    return shape_derivative

shapeDerivative_P = sdP(state, adjoint_P)

Similarly, computing the shape derivative of the inductance leads to

$$ 
 \mathrm{d}  L(\Omega)(\phi) =  \text{Re}\left( \frac{s l_z}{I^2} \int_\Omega \underline{\nu} A_\phi \nabla \underline{a} \cdot (\nabla \underline a)^* \; \mathrm{d} x  + \int_\Omega \underline{\nu} A_\phi \nabla \underline{a} \cdot \nabla \underline p_L^* \mathrm{d} x - \int_{\Omega_c} j\,\text{div} \phi\, \underline p_L^*  \mathrm{d} x \right) 
$$

In [72]:
def sdL(state : ngs.GridFunction,
        adjoint_L : ngs.GridFunction
        ) -> ngs.LinearForm :
    """ Compute the shape derivative (linear form) for the inductance"""

    # Finite element spaces
    mesh = state.space.mesh
    FEScomplex = ngs.H1(mesh, complex = True)**2 #ngs.VectorH1(mesh, complex=True)
    FESReal = ngs.H1(mesh)**2 # ngs.VectorH1(mesh)
    phi = FEScomplex.TestFunction()

    # Shorthands
    nu = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    coeff_induc = s * lz / (I**2) * nu.real
    dA = ngs.div(phi) * Id - ngs.grad(phi) - ngs.grad(phi).trans

    # Shape derivative 
    shape_derivative_cplx = ngs.LinearForm(FEScomplex)
    shape_derivative_cplx += coeff_induc * ngs.InnerProduct(dA * ngs.grad(state), ngs.grad(state)) * ngs.dx
    shape_derivative_cplx += -j * ngs.div(phi) * adjoint_L * ngs.dx
    shape_derivative_cplx += ngs.InnerProduct(nu * dA * ngs.grad(state), ngs.grad(adjoint_L)) * ngs.dx
    shape_derivative_cplx.Assemble()
    shape_derivative = ngs.LinearForm(FESReal)
    shape_derivative.vec.FV().NumPy()[:] = np.real(shape_derivative_cplx.vec.FV().NumPy()[:])

    return shape_derivative

shapeDerivative_L = sdP(state, adjoint_L)

### 4.b - Computation of a descent direction with Hilbertian regularization

- A descent direction can be extracted from the expression of the shape derivative. It is not unique!
- We can control the smoothness of the descent direction $d\in (H^1(\Omega))^2$ using a Hilbertian regularization; i.e. solve the follwing problem $\forall v \in (H^1(\Omega))^2 $
$$ B(d,v) = - dJ(v) $$
with $dJ$ the shape derivative of a given function $J$ and $A$ a positive definite; for instance
$ B(u,v) = r^2 \nabla u \cdot \nabla v + u \cdot v$, and $r$ a characteristic filtering radius.

In [ ]:
def hilbertianRegularization(sd : ngs.LinearForm,  # assembled shape derivative
                             r : float = 1.,
                             ) -> ngs.GridFunction : # descent direction
    
    fesVEC = ngs.VectorH1(
        sd.space.mesh,
        order=sd.space.globalorder,
        dirichlet="d",                  # we want the descent direction to be 0
        dirichletx="o|e|f|g|i|k|m|p",   # along certain directions
        dirichlety="a|b|c|n|j",
    )

    PHI, X = fesVEC.TnT()

    # H1 dot product
    B = ngs.BilinearForm(fesVEC)
    B += r * ngs.InnerProduct(ngs.grad(X), ngs.grad(PHI)) * ngs.dx 
    B += ngs.InnerProduct(X, PHI) * ngs.dx
    B.Assemble()

    gfX = ngs.GridFunction(fesVEC)
    gfX.vec.data = - B.mat.Inverse(fesVEC.FreeDofs()) * sd.vec
    return gfX


fesVEC = shapeDerivative_L.space
sd_lagrangian = ngs.GridFunction(fesVEC)
lam , beta = 1. , 1.
targetL = 1e-3
sd_lagrangian.vec.FV().NumPy()[:] = shapeDerivative_P.vec.FV().NumPy()[:] \
                                   + lam * shapeDerivative_L.vec.FV().NumPy()[:] / targetL\
                                   + beta * (inductance/targetL - 1) * shapeDerivative_L.vec.FV().NumPy()[:]/targetL

descentDirection = hilbertianRegularization(sd_lagrangian)

Draw(descentDirection, mesh, vectors={"grid_size": 40}, radius=0.02)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

________
## 5 - Gradient-based optimization

### 5.a - Optimization algorithm
We use an augmented Lagrangian algorithm to solve the constrained optimization problem. That is to say, we maximize with respect to a Lagrange multiplier $\lambda$ a sequence of inner minimization problem with respect to the optimization variables $y_{airgap}$ :

$$ \mathcal L(y_{airgap},\lambda) = P(y_{airgap}) + \lambda \left(\frac{L(y_{airgap})}{L_0} - 1 \right) + \frac{\beta}{2} \left(\frac{L(y_{airgap})}{L_0} -1 \right)^2 $$

# TO CONTINUE